# How you enforce to define methods

# Grab the hook of class creation

In [1]:
# library.py
class Base:
    def foo(self):
        return self.bar()

old_bc = __build_class__
def my_bc(*a, **kw):
    print('my build class -> ', a, kw)
    return old_bc(*a, **kw)

import builtins
builtins.__build_class__ = my_bc

In [2]:
# user.py
# from libraly import Base

class Derived(Base):
    def bar(self):
        return 'bar'

my build class ->  (<function Derived at 0x7fa15d766268>, 'Derived', <class '__main__.Base'>) {}


In [3]:
user = Derived()
user.bar()

'bar'

# Check whether bar is defind in library level

In [1]:
# library.py
class Base:
    def foo(self):
        return self.bar()

old_bc = __build_class__
def my_bc(fun, name, base=None, **kw):
    if base is Base:
        print('Check if bar method is defined')
    if base is not None:
        return old_bc(fun, name, base, **kw)
    return old_bc(fun, name)
    
import builtins
builtins.__build_class__ = my_bc

In [2]:
# user.py
# from libraly import Base

class Derived(Base):
    def bar(self):
        return 'bar'

Check if bar method is defined


In [3]:
user = Derived()
user.bar()

'bar'

## Metaclass

In [5]:
# library.py

class BaseMeta(type):
    def __new__(cls, name, bases, body):
        print('BaseMeta.__new__', cls, name, bases, body)
        return super().__new__(cls, name, bases, body)

class Base(metaclass=BaseMeta):
    def foo(self):
        return self.bar()

BaseMeta.__new__ <class '__main__.BaseMeta'> Base () {'__module__': '__main__', '__qualname__': 'Base', 'foo': <function Base.foo at 0x7ff7c0517950>}


## [`__new__`](https://docs.python.org/3/reference/datamodel.html#basic-customization)

[`**object.__new__(cls[, ...]`) **  ](https://docs.python.org/3/reference/datamodel.html#basic-customization)
```
Called to create a new instance of class cls. __new__() is a static method (special-cased so you need not declare 
it as such) that takes the class of which an instance was requested as its first argument. The remaining arguments are those passed to the object constructor expression (the call to the class). The return value of __new__() should be the new object instance (usually an instance of cls).

```


## If no bar method tell me!!

## ↓　bar がないのでエラーが出る

In [15]:
# library.py

class BaseMeta(type):
    def __new__(cls, name, bases, body):
        if not 'bar' in body:
            raise TypeError("Bad User Class")
        return super().__new__(cls, name, bases, body)

class Base(metaclass=BaseMeta):
    def foo(self):
        return self.bar()

TypeError: Bad User Class

↓　エラーが出ない

In [1]:
# library.py

class BaseMeta(type):
    def __new__(cls, name, bases, body):
        if not 'bar' in body:
            raise TypeError("Bad User Class")
        return super().__new__(cls, name, bases, body)

class Base(metaclass=BaseMeta):
    def foo(self):
        return self.bar()
    def bar(self):
        pass

## `__init_subclas__`を使う

In [14]:
# library.py


class Base:
    def foo(self):
        return self.bar()

    def __init_subclass__(cls, *args, **kwargs):
        print('init_subclass', cls, args, kwargs)
        if 'bar' not in cls.__dict__.keys():
            raise AttributeError('HEY!! bar method is not defind!!')

In [15]:
# user.py
# from libraly import Base

class Derived(Base):
    def bar(self):
        return 'bar'

init_subclass <class '__main__.Derived'> () {}


In [16]:
# user.py
# from libraly import Base

class Derived(Base):
    def yo(self):
        return 'bar'

init_subclass <class '__main__.Derived'> () {}


AttributeError: HEY!! bar method is not defind!!

## ABC Metaを使う

In [20]:
# library.py

from abc import ABCMeta
class Base(metaclass=ABCMeta):
    
    def foo(self):
        return self.bar()
    
    @abstractmethod
    def bar(self):
        pass

In [21]:
# user.py
# from libraly import Base

class Derived(Base):
    def yo(self):
        return 'bar'

In [22]:
user = Derived()

TypeError: Can't instantiate abstract class Derived with abstract methods bar

# decorator

## 関数を使う

In [20]:
# dec.py

from time import time


def timer(func, x, y):
    before = time()
    rv = func(x, y)
    after = time()
    print('elapsed', after - before)
    return rv


def add(x, y):
    return x + y


def sub(x, y):
    return x - y


print('add', timer(add, 10, 20))
print('sub', timer(sub, 10, 20))

elapsed 4.76837158203125e-07
add 30
elapsed 4.76837158203125e-07
sub -10


## 関数の中に関数を入れる

In [22]:
# dec.py

from time import time


def timer(func):
    def f(x, y):
        before = time()
        rv = func(x, y)
        after = time()
        print('elapsed', after - before)
        return rv
    return f


def add(x, y):
    return x + y

add = timer(add)

def sub(x, y):
    return x - y

sub = timer(sub)

print('add', add(10, 20))
print('sub', sub(10, 20))

elapsed 7.152557373046875e-07
add 30
elapsed 9.5367431640625e-07
sub -10


## 上と同じことをDecoratorで

In [24]:
# dec.py

from time import time


def timer(func):
    def f(x, y):
        before = time()
        rv = func(x, y)
        after = time()
        print('elapsed', after - before)
        return rv
    return f


@timer
def add(x, y):
    return x + y


@timer
def sub(x, y):
    return x - y


print('add', add(10, 20))
print('sub', sub(10, 20))

elapsed 7.152557373046875e-07
add 30
elapsed 7.152557373046875e-07
sub -10
